In [1]:
import torch
import os
# os.chdir('/content/Economy-and-Population-Growth')

In [ ]:
# !git clone https://github.com/GeorgeKontos14/Economy-and-Population-Growth.git

In [ ]:
# !git pull origin

In [2]:
torch.cuda.is_available()
from DataGeneration import generate_multiple_series, read_data
from Gibbs.Steps import *
from Gibbs.Parameters import step1_params, step2_params
from StoreData import *

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [3]:
generate = False
# generate = True
if generate:
    data = generate_multiple_series(150, 120, 10**4, 10**5, "gibbs.csv")
    w = np.random.random(120)
    w = w/np.sum(w)
    write_once(w, "weights.csv")
else:
    data = read_data("gibbs.csv")
    w = read_one_row("weights.csv")
w = torch.tensor(w, device=device)
n = 120
m = 25
l = 10
T = 150
q = 16
q_hat = q+15
burn_in = 1
draws = 3

In [4]:
X_path = "GibbsState/X_i.csv"
F_path = "GibbsState/F.csv"
S_m_path = "GibbsState/S_m.csv"
G_path = "GibbsState/G.csv"
H_path = "GibbsState/H.csv"
lambda_c_i_path = "GibbsState/Lambdas/c_i.csv"
lambda_g_j_path = "GibbsState/Lambdas/g_j.csv"
theta_c_i_path = "GibbsState/Thetas/c_i.csv"
theta_g_j_path = "GibbsState/Thetas/g_j.csv"
theta_h_k_path = "GibbsState/Thetas/h_k.csv"
kappa_c_i_path = "GibbsState/Kappas/c_i.csv"
kappa_g_j_path = "GibbsState/Kappas/g_j.csv"
kappa_h_k_path = "GibbsState/Kappas/h_k.csv"
J_path = "GibbsState/J.csv"
K_path = "GibbsState/K.csv"
p_c_lambda_path = "GibbsState/Ps/c_lambda.csv"
p_g_lambda_path = "GibbsState/Ps/g_lambda.csv"
p_c_theta_path = "GibbsState/Ps/c_theta.csv"
p_g_theta_path = "GibbsState/Ps/g_theta.csv"
p_h_theta_path = "GibbsState/Ps/h_theta.csv"
p_c_kappa_path = "GibbsState/Ps/c_kappa.csv"
p_g_kappa_path = "GibbsState/Ps/g_kappa.csv"
p_h_kappa_path = "GibbsState/Ps/h_kappa.csv"
sDa_path = "GibbsState/sDa.csv"
mu_c_path = "GibbsState/mu_c.csv"
sigma_m_path = "GibbsState/sigma_m.csv"
rho_m_path = "GibbsState/rho_m.csv"
omega_path = "GibbsState/omega.csv"
paths = [X_path, F_path, S_m_path, G_path, H_path, lambda_c_i_path, lambda_g_j_path, theta_c_i_path, theta_g_j_path, theta_h_k_path, kappa_c_i_path, kappa_g_j_path, kappa_h_k_path, J_path, K_path, p_c_lambda_path, p_g_lambda_path, p_c_theta_path, p_g_theta_path, p_h_theta_path, p_c_kappa_path, p_g_kappa_path, p_h_kappa_path, sDa_path, mu_c_path, sigma_m_path, rho_m_path, omega_path]

In [5]:
for path in paths:
    clear_csv(path)

In [6]:
X_list, F_list, S_m_list, G_list, H_list, l_c_i_list, l_g_j_list, theta_c_i_list, theta_g_j_list, theta_h_k_list, kappa_c_i_list, kappa_g_j_list, kappa_h_k_list, J_list, K_list, p_c_l_list, p_g_l_list, p_c_th_list, p_g_th_list, p_h_th_list, p_c_k_list, p_g_k_list, p_h_k_list, sDa_list, mu_c_list, sigma_m_list, rho_m_list, omega_list = run_Gibbs(data, n, m, l, w, T, q, q_hat, burn_in, draws, device)

In [7]:
store_Gibbs(X_list, F_list, S_m_list, G_list, H_list, l_c_i_list, l_g_j_list, theta_c_i_list, theta_g_j_list, theta_h_k_list, kappa_c_i_list, kappa_g_j_list, kappa_h_k_list, J_list, K_list, p_c_l_list, p_g_l_list, p_c_th_list, p_g_th_list, p_h_th_list, p_c_k_list, p_g_k_list, p_h_k_list, sDa_list, mu_c_list, sigma_m_list, rho_m_list, omega_list, X_path, F_path, S_m_path, G_path, H_path, lambda_c_i_path, lambda_g_j_path, theta_c_i_path, theta_g_j_path, theta_h_k_path, kappa_c_i_path, kappa_g_j_path, kappa_h_k_path, J_path, K_path, p_c_lambda_path, p_g_lambda_path, p_c_theta_path, p_g_theta_path, p_h_theta_path, p_c_kappa_path, p_g_kappa_path, p_h_kappa_path, sDa_path, mu_c_path, sigma_m_path, rho_m_path, omega_path)